In [2]:
import time
import requests
import pandas as pd
from selenium import webdriver
from webdriver_manager.chrome import ChromeDriverManager
from selenium.webdriver.chrome.service import Service
from selenium.webdriver.chrome.options import Options
from selenium.webdriver.common.by import By
from bs4 import BeautifulSoup
from selenium.webdriver.common.proxy import Proxy, ProxyType
import numpy as np
import aiohttp
import asyncio
import socks
import nest_asyncio
import random
import logging
import tenacity
from IPython.display import HTML
from fake_useragent import UserAgent
import itertools

# Парсинг сайта kinogo
# Алтухов Николай

In [5]:
# выбор метода парсинга, aiohttp или selenium
method = 'aiohttp'

In [1]:
# ссылки на страницы с жанрами
genres_urls = ['https://kinogo.uk/filmy-biografiya/',
               'https://kinogo.uk/filmy-boeviki/',
               "https://kinogo.uk/vesterny-online/",
               "https://kinogo.uk/voennye-filmy/",
               "https://kinogo.uk/smotret-detektivy/",
               "https://kinogo.uk/dokumentalnye-filmy/",
               "https://kinogo.uk/smotret-dramy/",
               "https://kinogo.uk/istoricheskie-filmy/",
               "https://kinogo.uk/smotret-komedii/",
               "https://kinogo.uk/smotret-priklyucheniya/",
               "https://kinogo.uk/semeynye-filmy/",
               'https://kinogo.uk/filmy-pro-sport/',
               "https://kinogo.uk/smotret-trillery/",
               "https://kinogo.uk/smotret-uzhasy/",
               "https://kinogo.uk/smotret-fantastiku/",
               "https://kinogo.uk/fantasy-online/"]


In [199]:
# Замер времени выполнения кода
start_time = time.time()

# Настройка логирования
logging.basicConfig(level=logging.INFO)
logger = logging.getLogger(__name__)

# Использование nest_asyncio для запуска асинхронного кода в jupyter lab
nest_asyncio.apply()


# Определение асинхронной функции для получения контента страницы с помощью aiohttp
@tenacity.retry(
    stop=tenacity.stop_after_attempt(3),
    wait=tenacity.wait_exponential(multiplier=1, min=2, max=6),
    retry=tenacity.retry_if_exception_type(aiohttp.ClientError)
)
async def fetch(session, url, ):
    headers = {
        'User-Agent': UserAgent().random,
        'Accept-Language': 'ru-RU,ru;q=0.9',
        'Accept': 'text/html,application/xhtml+xml,application/xml;q=0.9,image/webp,*/*;q=0.8',
        'Accept-Encoding': 'gzip, deflate, br',
        'DNT': '1'
    }
    proxy = "http://127.0.0.1:50445"
    async with session.get(url, headers=headers, proxy=proxy) as response:
        response.raise_for_status()  # Это вызовет исключение для ненулевых статусов
        return await response.text()

# Определение асинхронной функции для получения контента страницы с помощью Selenium
@tenacity.retry(
    stop=tenacity.stop_after_attempt(3),
    wait=tenacity.wait_exponential(multiplier=1, min=2, max=6),
    retry=tenacity.retry_if_exception_type(Exception))

async def fetch_with_selenium(url, proxy="127.0.0.1:50445"):
    chrome_options = Options()
    chrome_options.add_argument("--headless")
    chrome_options.add_argument("--disable-gpu")
    
    if proxy:
        chrome_options.add_argument(f'--proxy-server={proxy}')
        
    driver_path = 'C:\\Program Files\\Google\\Chrome\\Application\\chromedriver-win64\\chromedriver.exe'
    service = Service(driver_path)
    driver = webdriver.Chrome(service=service, options=chrome_options)

    try:
        driver.get(url)
        await asyncio.sleep(random.uniform(6, 12))  # Рандомная задержка
        html_content = driver.page_source
    finally:
        driver.quit()
    return html_content


# Определение асинхронной функции для парсинга контента страницы с помощью aiohttp
async def parse_with_aiohttp(url):
    async with aiohttp.ClientSession() as session:
        html_content = await fetch(session, url)
        await asyncio.sleep(random.uniform(2, 5))
        
        soup = BeautifulSoup(html_content, 'lxml')
        
        pages = soup.select_one('div.pagination__pages.d-flex.jc-center a:last-child')
        

        listings = []
        
            
        listings.append({'жанр': url, 'количество страниц': pages})
        
        return listings
    
    
# Определение асинхронной функции для парсинга контента страницы с помощью Selenium
async def parse_with_selenium(url):
    html_content = await fetch_with_selenium(url)
    soup = BeautifulSoup(html_content, 'lxml')
    items = soup.select('h2.card__title')
    rating_oblast1 = soup.select('div.iva-item-aside-GOesg')
    text1 = soup.select("p.styles-module-root-YczkZ.styles-module-size_s-xb_uK.styles-module-size_s_compensated-QmHFs.styles-module-size_s-_z7mI.styles-module-ellipsis-a2Uq1.stylesMarningNormal-module-root-S7NIr.stylesMarningNormal-module-paragraph-s-Yhr2e.styles-module-noAccent-LowZ8.styles-module-root_bottom-G4JNz.styles-module-margin-bottom_6-_aVZm")
    gorod1 = soup.select("p.styles-module-root-YczkZ.styles-module-size_s-xb_uK.styles-module-size_s_compensated-QmHFs.styles-module-size_s-_z7mI.styles-module-ellipsis-a2Uq1.styles-module-ellipsis_oneLine-swQCL.stylesMarningNormal-module-root-S7NIr.stylesMarningNormal-module-paragraph-s-Yhr2e.styles-module-noAccent-LowZ8.styles-module-root_top-p0_50.styles-module-margin-top_0-Mk_hC")
    listings = []
    for idx, item in enumerate(items):
        title = item.get_text(strip=True)
        link = item['href']
        
        # Проверяем, что элементы rating_oblast1, text1 и gorod1 существуют и добавляем их в listings
        rating_oblast = rating_oblast1[idx].get_text(strip=True) if idx < len(rating_oblast1) else None
        text = text1[idx].get_text(strip=True) if idx < len(text1) else None
        gorod = gorod1[idx].get_text(strip=True) if idx < len(gorod1) else None
        
        listings.append({'title': title, 'link': link, 'rating_oblast1': rating_oblast, 'text1': text, 'gorod1': gorod})
    return listings

# Определение асинхронной функции для выполнения парсинга в пакетном режиме
async def batch_executor(urls, batch_size=2, parse_func=None):
    results = []
    for i in range(0, len(urls), batch_size):
        batch = urls[i:i + batch_size]
        tasks = [parse_func(url) for url in batch]
        batch_results = await asyncio.gather(*tasks, return_exceptions=True)
        for result in batch_results:
            if isinstance(result, Exception):
                logging.error(f"Произошла ошибка: {result}")
            else:
                results.extend(result)
    return results

# Определение асинхронной функции для выполнения парсинга всех страниц
async def main(urls, parse_func):
    all_listings = await batch_executor(urls, batch_size=1, parse_func=parse_func)  # Указываем размер партии
    df = pd.DataFrame(all_listings)
    return df



# Выполнение парсинга с использованием aiohttp или Selenium в зависимости от значения параметра method
if method =='aiohttp':
    df = await main(genres_urls, parse_func=parse_with_aiohttp)
else:
    logging.getLogger('selenium.webdriver').setLevel(logging.WARNING) # будут записываться только сообщения с уровнем WARNING и выше
    df = await main(genres_urls, parse_func=parse_with_selenium)


print(f"Время выполнения функции: {time.time() - start_time} секунд")

Время выполнения функции: 67.5973539352417 секунд


In [ ]:
df__ = df.copy()

In [211]:
df__

,жанр,количество страниц
0,https://kinogo.uk/filmy-biografiya/,[101]
1,https://kinogo.uk/filmy-boeviki/,[337]
2,https://kinogo.uk/vesterny-online/,[25]
3,https://kinogo.uk/voennye-filmy/,[94]
4,https://kinogo.uk/smotret-detektivy/,[206]
5,https://kinogo.uk/dokumentalnye-filmy/,[59]
6,https://kinogo.uk/smotret-dramy/,[915]
7,https://kinogo.uk/istoricheskie-filmy/,[101]
8,https://kinogo.uk/smotret-komedii/,[579]
9,https://kinogo.uk/smotret-priklyucheniya/,[257]


In [249]:
df = df__.copy()
df['количество страниц'] = df['количество страниц'].astype(str).str.extract(r'>(\d+)<').astype(int) # оставить только числа в столбце с количеством страниц

In [251]:
df

,жанр,количество страниц
0,https://kinogo.uk/filmy-biografiya/,101
1,https://kinogo.uk/filmy-boeviki/,337
2,https://kinogo.uk/vesterny-online/,25
3,https://kinogo.uk/voennye-filmy/,94
4,https://kinogo.uk/smotret-detektivy/,206
5,https://kinogo.uk/dokumentalnye-filmy/,59
6,https://kinogo.uk/smotret-dramy/,915
7,https://kinogo.uk/istoricheskie-filmy/,101
8,https://kinogo.uk/smotret-komedii/,579
9,https://kinogo.uk/smotret-priklyucheniya/,257


In [255]:
df['все ссылки'] = df.apply(lambda row: [f"{row['жанр']}page/{i}" for i in range(1, row['количество страниц']+1)], axis=1) # определение всех необходимых ссылок на основе количества страниц и ссылки жанров

In [276]:
flat_list = list(itertools.chain(*df['все ссылки'].tolist())) # раскрытие списков в списке

['https://kinogo.uk/filmy-biografiya/page/2',
 'https://kinogo.uk/filmy-biografiya/page/3']

In [307]:
urls = flat_list

In [323]:
# Замер времени выполнения кода
start_time = time.time()
# Настройка логирования
logging.basicConfig(level=logging.INFO)
logger = logging.getLogger(__name__)
# Использование nest_asyncio для запуска асинхронного кода в ноутбуке Jupyter
nest_asyncio.apply()


# Определение асинхронной функции для получения контента страницы с помощью aiohttp
@tenacity.retry(
    stop=tenacity.stop_after_attempt(3),
    wait=tenacity.wait_exponential(multiplier=1, min=2, max=6),
    retry=tenacity.retry_if_exception_type(aiohttp.ClientError)
)
async def fetch(session, url, ):
    headers = {
        'User-Agent': UserAgent().random,
        'Accept-Language': 'ru-RU,ru;q=0.9',
        'Accept': 'text/html,application/xhtml+xml,application/xml;q=0.9,image/webp,*/*;q=0.8',
        'Accept-Encoding': 'gzip, deflate, br',
        'DNT': '1'
    }
    proxy = "http://127.0.0.1:50445"
    async with session.get(url, headers=headers, proxy=proxy) as response:
        response.raise_for_status()  # Это вызовет исключение для ненулевых статусов
        return await response.text()

# Определение асинхронной функции для получения контента страницы с помощью Selenium
@tenacity.retry(
    stop=tenacity.stop_after_attempt(3),
    wait=tenacity.wait_exponential(multiplier=1, min=2, max=6),
    retry=tenacity.retry_if_exception_type(Exception))

async def fetch_with_selenium(url, proxy="127.0.0.1:50445"):
    chrome_options = Options()
    chrome_options.add_argument("--headless")
    chrome_options.add_argument("--disable-gpu")
    
    if proxy:
        chrome_options.add_argument(f'--proxy-server={proxy}')
        
    driver_path = 'C:\\Program Files\\Google\\Chrome\\Application\\chromedriver-win64\\chromedriver.exe'
    service = Service(driver_path)
    driver = webdriver.Chrome(service=service, options=chrome_options)

    try:
        driver.get(url)
        await asyncio.sleep(random.uniform(6, 12))  # Рандомная задержка
        html_content = driver.page_source
    finally:
        driver.quit()
    return html_content

# Определение асинхронной функции для парсинга контента страницы с помощью aiohttp
async def parse_with_aiohttp(url):
    async with aiohttp.ClientSession() as session:
        html_content = await fetch(session, url)
        await asyncio.sleep(random.uniform(2, 5))
        
        soup = BeautifulSoup(html_content, 'lxml')
        
        items = soup.select('h2.card__title a')
        year = soup.select('ul.card__list li:nth-of-type(1) a')
        country = soup.select('ul.card__list li:nth-of-type(2) a')
        genre = soup.select('ul.card__list li:nth-of-type(3) a')
        text = soup.select('p.card__text.line-clamp')
        ratingsite = soup.select('.current-rating')

        
        
        listings = []
        for idx, item in enumerate(items):
            title = item.get_text(strip=True)
            link = item['href']
            
            year1 = year[idx].get_text(strip=True) if idx < len(year) else None
            country1 = country[idx].get_text(strip=True) if idx < len(country) else None
            genre1 = genre[idx].get_text(strip=True) if idx < len(genre) else None
            text1 = text[idx].get_text(strip=True) if idx < len(text) else None
            ratingsite1 = ratingsite[idx].get_text(strip=True) if idx < len(ratingsite) else None
            
            listings.append({'Название': title, 'Ссылка': link, 'Год': year1, 'Страна выпуска': country1, 'Жанры': genre1, 'Описание':text1, 'Рейтинг':ratingsite1})
        
        return listings
    
# Определение асинхронной функции для парсинга контента страницы с помощью Selenium
async def parse_with_selenium(url):
    html_content = await fetch_with_selenium(url)
    soup = BeautifulSoup(html_content, 'lxml')
    items = soup.select('h2.card__title')
    rating_oblast1 = soup.select('div.iva-item-aside-GOesg')
    text1 = soup.select("p.styles-module-root-YczkZ.styles-module-size_s-xb_uK.styles-module-size_s_compensated-QmHFs.styles-module-size_s-_z7mI.styles-module-ellipsis-a2Uq1.stylesMarningNormal-module-root-S7NIr.stylesMarningNormal-module-paragraph-s-Yhr2e.styles-module-noAccent-LowZ8.styles-module-root_bottom-G4JNz.styles-module-margin-bottom_6-_aVZm")
    gorod1 = soup.select("p.styles-module-root-YczkZ.styles-module-size_s-xb_uK.styles-module-size_s_compensated-QmHFs.styles-module-size_s-_z7mI.styles-module-ellipsis-a2Uq1.styles-module-ellipsis_oneLine-swQCL.stylesMarningNormal-module-root-S7NIr.stylesMarningNormal-module-paragraph-s-Yhr2e.styles-module-noAccent-LowZ8.styles-module-root_top-p0_50.styles-module-margin-top_0-Mk_hC")
    listings = []
    for idx, item in enumerate(items):
        title = item.get_text(strip=True)
        link = item['href']
        
        # Проверяем, что элементы rating_oblast1, text1 и gorod1 существуют и добавляем их в listings
        rating_oblast = rating_oblast1[idx].get_text(strip=True) if idx < len(rating_oblast1) else None
        text = text1[idx].get_text(strip=True) if idx < len(text1) else None
        gorod = gorod1[idx].get_text(strip=True) if idx < len(gorod1) else None
        
        listings.append({'title': title, 'link': link, 'rating_oblast1': rating_oblast, 'text1': text, 'gorod1': gorod})
    return listings

# Определение асинхронной функции для выполнения парсинга в пакетном режиме
async def batch_executor(urls, batch_size=2, parse_func=None):
    results = []
    for i in range(0, len(urls), batch_size):
        batch = urls[i:i + batch_size]
        tasks = [parse_func(url) for url in batch]
        batch_results = await asyncio.gather(*tasks, return_exceptions=True)
        for result in batch_results:
            if isinstance(result, Exception):
                logging.error(f"Произошла ошибка: {result}")
            else:
                results.extend(result)
    return results

# Определение асинхронной функции для выполнения парсинга всех страниц
async def main(urls, parse_func):
    all_listings = await batch_executor(urls, batch_size=1, parse_func=parse_func)  # Указываем размер партии
    df = pd.DataFrame(all_listings)
    return df

# Выполнение парсинга с использованием aiohttp или Selenium в зависимости от значения параметра method
async def run(urls, method):
    try:
        start_time = time.time()

        if method == 'aiohttp':
            df = await main(urls, parse_func=parse_with_aiohttp)
        elif method == 'selenium':
            logging.getLogger('selenium.webdriver').setLevel(logging.WARNING)
            df = await main(urls, parse_func=parse_with_selenium)

        elapsed_time = time.time() - start_time
        print(f"Execution time: {elapsed_time:.2f} seconds")

        return df
    except Exception as e:
        logger.error(f"Произошла ошибка в функции run: {e}")
        raise

if __name__ == "__main__":
    df = asyncio.run(run(urls, method))
    

    
    
print(f"Время выполнения функции: {time.time() - start_time} секунд")

Execution time: 9.22 seconds
Время выполнения функции: 9.217278480529785 секунд


In [298]:
df.to_excel('ok.xlsx')

In [7]:
df = pd.read_excel('ok.xlsx')
dfcopy = df.copy()

In [9]:
df_without_dupl = dfcopy.drop_duplicates(subset=['Название','Ссылка'])

In [53]:
urls = df_without_dupl['Ссылка'][10000:].to_list()

In [55]:
# Замер времени выполнения кода
start_time = time.time()
# Настройка логирования
logging.basicConfig(level=logging.INFO)
logger = logging.getLogger(__name__)
# Использование nest_asyncio для запуска асинхронного кода в ноутбуке Jupyter
nest_asyncio.apply()


# Определение асинхронной функции для получения контента страницы с помощью aiohttp
@tenacity.retry(
    stop=tenacity.stop_after_attempt(3),
    wait=tenacity.wait_exponential(multiplier=1, min=2, max=6),
    retry=tenacity.retry_if_exception_type(aiohttp.ClientError)
)
async def fetch(session, url, ):
    headers = {
        'User-Agent': UserAgent().random,
        'Accept-Language': 'ru-RU,ru;q=0.9',
        'Accept': 'text/html,application/xhtml+xml,application/xml;q=0.9,image/webp,*/*;q=0.8',
        'Accept-Encoding': 'gzip, deflate, br',
        'DNT': '1'
    }
    proxy = "http://127.0.0.1:50445"
    async with session.get(url, headers=headers, proxy=proxy) as response:
        response.raise_for_status()  # Это вызовет исключение для ненулевых статусов
        return await response.text()

# Определение асинхронной функции для получения контента страницы с помощью Selenium
@tenacity.retry(
    stop=tenacity.stop_after_attempt(3),
    wait=tenacity.wait_exponential(multiplier=1, min=2, max=6),
    retry=tenacity.retry_if_exception_type(Exception))

async def fetch_with_selenium(url, proxy="127.0.0.1:50445"):
    chrome_options = Options()
    chrome_options.add_argument("--headless")
    chrome_options.add_argument("--disable-gpu")
    
    if proxy:
        chrome_options.add_argument(f'--proxy-server={proxy}')
        
    driver_path = 'C:\\Program Files\\Google\\Chrome\\Application\\chromedriver-win64\\chromedriver.exe'
    service = Service(driver_path)
    driver = webdriver.Chrome(service=service, options=chrome_options)

    try:
        driver.get(url)
        await asyncio.sleep(random.uniform(6, 12))  # Рандомная задержка
        html_content = driver.page_source
    finally:
        driver.quit()
    return html_content

# Определение асинхронной функции для парсинга контента страницы с помощью aiohttp
async def parse_with_aiohttp(url):
    async with aiohttp.ClientSession() as session:
        html_content = await fetch(session, url)
        await asyncio.sleep(random.uniform(1, 3))
        
        soup = BeautifulSoup(html_content, 'lxml')
        listings = []
        def get_text_or_none(selector):
            element = soup.select_one(selector)
            return element.get_text(strip=True) if element else None
        
        title_ru = get_text_or_none('.page__header h1[itemprop="name"]')
        title_orig = get_text_or_none('.page__header .pmovie__original-title')
        year = get_text_or_none('ul.pmovie__header-list.flex-grow-1 li:nth-of-type(1)')
        country = get_text_or_none('ul.pmovie__header-list.flex-grow-1 li:nth-of-type(2)')
        genre = get_text_or_none('ul.pmovie__header-list.flex-grow-1 li:nth-of-type(3)')
        film_director = get_text_or_none('ul.pmovie__header-list.flex-grow-1 li:nth-last-of-type(2)')
        actors = get_text_or_none('ul.pmovie__header-list.flex-grow-1 li:nth-last-of-type(1)')
        text = get_text_or_none('div.page__text.full-text.clearfix')
        ratingsite = get_text_or_none('.card__rating-ext-count.centered-content')
        ratingimdb = get_text_or_none('.card__rating-ext.imdb')
        ratingkp = get_text_or_none('.card__rating-ext.kp')
        

        listings.append({'Ссылка':url,'Название': title_ru, 'Ориг. название': title_orig, 'Год': year, 'Страна выпуска': country,
                         'Жанры': genre, 'Описание':text,'Режисер':film_director,'Актеры':actors, 'Рейтинг':ratingsite,
                        "Рейтинг IMDB":ratingimdb, "Рейтинг Кинопоиск":ratingkp})
        
        return listings
    
# Определение асинхронной функции для парсинга контента страницы с помощью Selenium
async def parse_with_selenium(url):
    html_content = await fetch_with_selenium(url)
    soup = BeautifulSoup(html_content, 'lxml')
    items = soup.select('h2.card__title')
    rating_oblast1 = soup.select('div.iva-item-aside-GOesg')
    text1 = soup.select("p.styles-module-root-YczkZ.styles-module-size_s-xb_uK.styles-module-size_s_compensated-QmHFs.styles-module-size_s-_z7mI.styles-module-ellipsis-a2Uq1.stylesMarningNormal-module-root-S7NIr.stylesMarningNormal-module-paragraph-s-Yhr2e.styles-module-noAccent-LowZ8.styles-module-root_bottom-G4JNz.styles-module-margin-bottom_6-_aVZm")
    gorod1 = soup.select("p.styles-module-root-YczkZ.styles-module-size_s-xb_uK.styles-module-size_s_compensated-QmHFs.styles-module-size_s-_z7mI.styles-module-ellipsis-a2Uq1.styles-module-ellipsis_oneLine-swQCL.stylesMarningNormal-module-root-S7NIr.stylesMarningNormal-module-paragraph-s-Yhr2e.styles-module-noAccent-LowZ8.styles-module-root_top-p0_50.styles-module-margin-top_0-Mk_hC")
    listings = []
    for idx, item in enumerate(items):
        title = item.get_text(strip=True)
        link = item['href']
        
        # Проверяем, что элементы rating_oblast1, text1 и gorod1 существуют и добавляем их в listings
        rating_oblast = rating_oblast1[idx].get_text(strip=True) if idx < len(rating_oblast1) else None
        text = text1[idx].get_text(strip=True) if idx < len(text1) else None
        gorod = gorod1[idx].get_text(strip=True) if idx < len(gorod1) else None
        
        listings.append({'title': title, 'link': link, 'rating_oblast1': rating_oblast, 'text1': text, 'gorod1': gorod})
    return listings

# Определение асинхронной функции для выполнения парсинга в пакетном режиме
async def batch_executor(urls, batch_size=2, parse_func=None):
    results = []
    for i in range(0, len(urls), batch_size):
        batch = urls[i:i + batch_size]
        tasks = [parse_func(url) for url in batch]
        batch_results = await asyncio.gather(*tasks, return_exceptions=True)
        for result in batch_results:
            if isinstance(result, Exception):
                logging.error(f"Произошла ошибка: {result}")
            else:
                results.extend(result)
    return results

# Определение асинхронной функции для выполнения парсинга всех страниц
async def main(urls, parse_func):
    all_listings = await batch_executor(urls, batch_size=1, parse_func=parse_func)  # Указываем размер партии
    df = pd.DataFrame(all_listings)
    return df


# Выполнение парсинга с использованием aiohttp или Selenium в зависимости от значения параметра method
async def run(urls, method):
    try:
        start_time = time.time()

        if method == 'aiohttp':
            df = await main(urls, parse_func=parse_with_aiohttp)
        elif method == 'selenium':
            logging.getLogger('selenium.webdriver').setLevel(logging.WARNING)
            df = await main(urls, parse_func=parse_with_selenium)

        elapsed_time = time.time() - start_time
        print(f"Execution time: {elapsed_time:.2f} seconds")

        return df
    except Exception as e:
        logger.error(f"Произошла ошибка в функции run: {e}")
        raise

if __name__ == "__main__":
    df = asyncio.run(run(urls, method))
    

    
    
    
    
    
print(f"Время выполнения функции: {time.time() - start_time} секунд")

Execution time: 29623.20 seconds
Время выполнения функции: 29623.19811797142 секунд


In [56]:
df

,Ссылка,Название,Ориг. название,Год,Страна выпуска,Жанры,Описание,Режисер,Актеры,Рейтинг,Рейтинг IMDB,Рейтинг Кинопоиск
0,https://kinogo.uk/10782-delikatesy-1990.html,Деликатесы (1990),Delicatessen,Год выпуска:1990,Страна:Франция,"Жанр:Фильмы,Драма,Комедия,Криминал,Ужасы","""Пища души"" - это фильм, события которого разв...","Режиссер:Марк Каро, Жан-Пьер Жёне","В ролях:Паскаль Бенезек, Доминик Пинон, Мари-Л...",0,7.6(89 959),7.2(11 126)
1,https://kinogo.uk/11618-vozvraschenie-2006.html,Возвращение (2006),Volver,Год выпуска:2006,Страна:Испания,"Жанр:Фильмы,Драма,Комедия",Эта картина ведет нас через таинственную истор...,Режиссер:Педро Альмодовар,"В ролях:Пенелопа Крус, Кармен Маура, Лола Дуэн...",0,7.6(105777),7.7(28 653)
2,https://kinogo.uk/7177-jeffekt-babochki-2003.html,Эффект бабочки (2003),The Butterfly Effect,Год выпуска:2003,"Страна:США,Канада","Жанр:Фильмы,Драма,Триллер,Фантастика",Молодой Эван страдает от загадочного заболеван...,"Режиссер:Эрик Бресс, Дж. Макки Грубер","В ролях:Эштон Кутчер, Эми Смарт, Элден Хенсон,...",4.4,7.6(504249),8.2(507 228)
3,https://kinogo.uk/12032-korol-drifta-opasnyj-z...,Король дрифта: Опасный заезд (2022),Alivehoon,Год выпуска:2022,Страна:Япония,"Жанр:Фильмы,Драма,Спорт","Коичи – имя, которое знают все поклонники кибе...",Режиссер:Тэн Симояма,"В ролях:Сюхэй Номура, Аи Ёсикава, Таканори Дзи...",5,8.0(146),None
4,https://kinogo.uk/7174-koldovstvo-novyj-ritual...,Колдовство: Новый ритуал (2020),The Craft: Legacy,Год выпуска:2020,Страна:США,"Жанр:Фильмы,Драма,Ужасы,Фэнтези",В центре сюжета нашей истории - юная старшекла...,Режиссер:Зои Листер Джонс,"В ролях:Кейли Спейни, Гидеон Адлон, Лови Симон...",3.9,4.4(16 476),4.8(13 614)
...,...,...,...,...,...,...,...,...,...,...,...,...
11117,https://kinogo.uk/17603-baba-jaga-protiv-1979....,Баба Яга против! (1 сезон),None,Год выпуска:1979,Страна:СССР,"Жанр:Мультфильмы,Фэнтези","В далекой волшебной стране, где легенды оживаю...",Режиссер:Владимир Пекарь,"В ролях:Ольга Аросева, Гарри Бардин",1,6.7(183),6.9(24146)
11118,https://kinogo.uk/17697-poslednjaja-missija-an...,Последняя миссия ангела: Любовь (1 сезон),"Dan, hanaui sarang",Год выпуска:2019,Страна:Корея Южная,"Жанр:Сериалы,Фэнтези","В сонме небесных светил, где каждый ангел обер...",Режиссер:Ли Джон-соп,"В ролях:Щин Хе-сон, Ким Мён-су, Ли Дон-гон, Ки...",0,7.8(2892),7.8(17528)
11119,https://kinogo.uk/15380-arijetti-iz-strany-lil...,Ариэтти из страны лилипутов (2010),Karigurashi no Arietti,Год выпуска:2010,Страна:Япония,"Жанр:Аниме,Фэнтези","В тихом уголке мира, где жизнь текла своим чер...",Режиссер:Хиромаса Ёнэбаяси,"В ролях:Мираи Сида, Рюносукэ Камики, Тацуя Фуд...",5,7.6(104112),7.9(51 305)
11120,https://kinogo.uk/16169-rozhdestvenskie-tufelk...,Рождественские туфельки (2018),A Shoe Addict's Christmas,Год выпуска:2018,Страна:США,"Жанр:Фильмы,Фэнтези","Ноэль, занимающая должность менеджера по персо...",Режиссер:Майкл Робисон,"В ролях:Кендес Камерон Буре, Люк МакФарлейн, Д...",0,6.7(2809),6.8(8319)


In [57]:
df_10000_22000 = df.copy()

In [58]:
df_10000_22000.to_excel('films_10000_22000.xlsx')

In [47]:
df_0_10000 = df.copy()

In [51]:
df_0_10000.to_excel('films_0_10000.xlsx')

In [71]:
all_films = pd.concat([df_0_10000,df_10000_22000], axis=0)

In [73]:
all_films.to_excel('films.xlsx', index=False)

In [87]:
all_films = pd.read_excel('films.xlsx')

In [93]:
all_films = all_films.reset_index(drop=True)

In [95]:
all_films

,Ссылка,Название,Ориг. название,Год,Страна выпуска,Жанры,Описание,Режисер,Актеры,Рейтинг,Рейтинг IMDB,Рейтинг Кинопоиск
0,https://kinogo.uk/22795-ja-selin-dion-2024.html,Я – Селин Дион (2024),I Am: Celine Dion,Год выпуска:2024,"Страна:Канада,США","Жанр:Фильмы,Биография,Документальный",Фильм погружает зрителей в повседневную жизнь ...,Режиссер:Ирен Тейлор Бродски,"В ролях:Селин Дион, Nelson Angélil, Eddy Angél...",4.3,8.1(2490),NaN
1,https://kinogo.uk/22737-stat-karlom-lagerfeldo...,Стать Карлом Лагерфельдом (1 сезон),Becoming Karl Lagerfeld,Год выпуска:2024,Страна:Франция,"Жанр:Сериалы,Биография,Драма","Сериал, основанный на бестселлере Рафаэля Бакк...","Режиссер:Одри Эструго, Жером Салль","В ролях:Даниэль Брюль, Теодор Пеллерен, Алекс ...",4.1,7.3(1309),7.043(337)
2,https://kinogo.uk/22458-trudnye-mili-2023.html,Трудные мили (2023),Hard Miles,Год выпуска:2023,Страна:США,"Жанр:Фильмы,Биография,Драма,Спорт","Решительный социальный работник, посвятивший с...",Режиссер:Р.Дж. Дэниэл Ханна,"В ролях:Мэттью Модайн, Шон Эстин, Лесли Дэвид ...",2.8,6.8,NaN
3,https://kinogo.uk/22358-kassandro-2023.html,Кассандро (2023),Cassandro,Год выпуска:2023,"Страна:Мексика,США","Жанр:Фильмы,Биография,Драма,Спорт","История Кассандро начинается в Эль-Пасо, где м...",Режиссер:Роджер Росс Уильямс,"В ролях:Гаэль Гарсиа Берналь, Роберта Колиндре...",3.7,6.3,NaN
4,https://kinogo.uk/21215-odna-zhizn-2023.html,Одна жизнь (2023),One Life,Год выпуска:2023,Страна:Великобритания,"Жанр:Фильмы,Биография,Драма,История",Скромного и непубличного джентльмена Николаса ...,Режиссер:Джеймс Хоуз,"В ролях:Энтони Хопкинс, Хелена Бонем Картер, Д...",4.4,7.6,8.0
...,...,...,...,...,...,...,...,...,...,...,...,...
21117,https://kinogo.uk/17603-baba-jaga-protiv-1979....,Баба Яга против! (1 сезон),NaN,Год выпуска:1979,Страна:СССР,"Жанр:Мультфильмы,Фэнтези","В далекой волшебной стране, где легенды оживаю...",Режиссер:Владимир Пекарь,"В ролях:Ольга Аросева, Гарри Бардин",1.0,6.7(183),6.9(24146)
21118,https://kinogo.uk/17697-poslednjaja-missija-an...,Последняя миссия ангела: Любовь (1 сезон),"Dan, hanaui sarang",Год выпуска:2019,Страна:Корея Южная,"Жанр:Сериалы,Фэнтези","В сонме небесных светил, где каждый ангел обер...",Режиссер:Ли Джон-соп,"В ролях:Щин Хе-сон, Ким Мён-су, Ли Дон-гон, Ки...",0.0,7.8(2892),7.8(17528)
21119,https://kinogo.uk/15380-arijetti-iz-strany-lil...,Ариэтти из страны лилипутов (2010),Karigurashi no Arietti,Год выпуска:2010,Страна:Япония,"Жанр:Аниме,Фэнтези","В тихом уголке мира, где жизнь текла своим чер...",Режиссер:Хиромаса Ёнэбаяси,"В ролях:Мираи Сида, Рюносукэ Камики, Тацуя Фуд...",5.0,7.6(104112),7.9(51 305)
21120,https://kinogo.uk/16169-rozhdestvenskie-tufelk...,Рождественские туфельки (2018),A Shoe Addict's Christmas,Год выпуска:2018,Страна:США,"Жанр:Фильмы,Фэнтези","Ноэль, занимающая должность менеджера по персо...",Режиссер:Майкл Робисон,"В ролях:Кендес Камерон Буре, Люк МакФарлейн, Д...",0.0,6.7(2809),6.8(8319)
